# Folium ile Choropleth Haritası Oluşturma

## Kütüphaneleri Çağırma

In [1]:
import folium 
import pandas as pd
import geopandas as gpd
import requests as rqs

## Covid Verilerini https://www.worldometers.info/coronavirus/ sitesinde okuma

In [2]:
data=rqs.get('https://www.worldometers.info/coronavirus/')
data_corona=pd.read_html(data.text)

In [3]:
for data_covid in data_corona:
    print(data_covid)

             Country,Other  TotalCases NewCases  TotalDeaths NewDeaths  \
0                    World     1363099  +17,095      76370.0    +1,716   
1                      USA      368174   +1,170      10966.0       +95   
2                    Spain      140510   +3,835      13798.0      +457   
3                    Italy      132547      NaN      16523.0       NaN   
4                  Germany      103717     +342       1822.0       +12   
..                     ...         ...      ...          ...       ...   
208  Saint Pierre Miquelon           1      NaN          NaN       NaN   
209            South Sudan           1      NaN          NaN       NaN   
210            Timor-Leste           1      NaN          NaN       NaN   
211                  China       81740      +32       3331.0       NaN   
212                 Total:     1363099  +17,095      76370.0    +1,716   

     TotalRecovered  ActiveCases  Serious,Critical  Tot Cases/1M pop  \
0          293743.0       992986       

## Verinin istenilen sütunlarını aktarma

In [4]:
data_covid=data_covid[['Country,Other','TotalCases']]
data_covid.head()

,"Country,Other",TotalCases
0,World,1346004
1,China,81708
2,USA,367004
3,Spain,136675
4,Italy,132547


## Folium altlık haritasını çağırma

In [5]:
m=folium.Map()
m

## Dünya ülkelerini temsil eden GeoJSON verisini okuma

In [6]:
url = 'https://raw.githubusercontent.com/python-visualization/folium/master/examples/data'
country_shapes = f'{url}/world-countries.json'

## Covid verisi ile GeoJSON dosyası arasındaki isim farklarını giderme

In [8]:
data_covid.replace('USA', "United States of America", inplace = True)
data_covid.replace('Tanzania', "United Republic of Tanzania", inplace = True)
data_covid.replace('Democratic Republic of Congo', "Democratic Republic of the Congo", inplace = True)
data_covid.replace('Congo', "Republic of the Congo", inplace = True)
data_covid.replace('Lao', "Laos", inplace = True)
data_covid.replace('Syrian Arab Republic', "Syria", inplace = True)
data_covid.replace('Serbia', "Republic of Serbia", inplace = True)
data_covid.replace('Czechia', "Czech Republic", inplace = True)
data_covid.replace('UAE', "United Arab Emirates", inplace = True)

## Toplam Vaka Sayısına Göre Choropleth haritası oluşturma

In [9]:
folium.Choropleth(
        geo_data=country_shapes,
        name='Choropleth Covid-19',
        data=data_covid,
        columns=['Country,Other','TotalCases'],
        key_on='feature.properties.name',
        fill_color='RdPu',
        nan_fill_color='white'
).add_to(m)
m

## fill_color='' için kullanılabilecek diğer renkler: 
#### 'BuGn', 'BuPu', 'GnBu', 'OrRd', 'PuBu', 'PuBuGn', 'PuRd', 'RdPu','YlGn', 'YlGnBu', 'YlOrBr', and 'YlOrRd'.

## Ülkelerin enlem ve boylam bilgisini içeren excel dosyasını okuma

In [10]:
ülke=pd.read_excel("ülke.xlsx")
ülke.head()

,ülke,enlem,boylam,isim,Unnamed: 4,Unnamed: 5
0,AD,42.546245,1.601554,Andorra,NaN,NaN
1,AE,23.424076,53.847818,Birleşik Arap Emirlikleri,NaN,NaN
2,AF,33.939110,67.709953,Afganistan,NaN,NaN
3,AG,17.060816,-61.796428,Antigua ve Barbuda,NaN,NaN
4,AI,18.220554,-63.068615,Anguilla,NaN,NaN


## Enlem ve Boylam sütunlarından Nan değerlerini kaldırma

In [11]:
ülke=ülke.dropna(subset=['enlem'])
ülke=ülke.dropna(subset=['boylam'])

## Covid verileri ile ülke excel dosyasını birleştirme

In [13]:
data_covid.rename(columns={'Country,Other':'isim'},inplace=True)
combine=data_covid.merge(ülke,on='isim')
combine.head()

,isim,TotalCases,ülke,enlem,boylam,Unnamed: 4,Unnamed: 5
0,China,81708,CN,35.861660,104.195397,NaN,NaN
1,United States of America,367004,BİZE,37.090240,-95.712891,NaN,Cayman Islands
2,Spain,136675,ES,40.463667,-3.749220,NaN,NaN
3,Italy,132547,O,41.871940,12.567380,NaN,NaN
4,Germany,103375,DE,51.165691,10.451526,NaN,NaN


## Haritaya işaretçi ekleme

In [28]:
for lat, lon,isim,totalcases in zip(combine['enlem'],combine['boylam'],combine['isim'],combine['TotalCases']):
     folium.Circle(
        radius=250,
        color='blue',
        location=[lat, lon],
        popup=(isim,totalcases),
    ).add_to(m)
m